In [1]:
import pandas as pd
import numpy as np
from station import Station
%load_ext autoreload
%autoreload

In [2]:
# read in data files
ride_df = pd.read_csv('data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals.csv')
map_df = pd.read_csv('data/CTA_-_System_Information_-_List_of__L__Stops.csv')

In [3]:
station_list = ride_df.stationname.unique().tolist()
sumlist = []
for station in station_list:
    print(station)
    stat = Station(station, ride_df, map_df)
    sumlist.append(stat.summary)
station_summary = pd.DataFrame(sumlist)    

UIC-Halsted
['U', 'W', 'A']
Halsted-Orange
['U', 'W', 'A']
Granville
['U', 'W', 'A']
Jackson/Dearborn
['U', 'W', 'A']
Damen-Brown
['U', 'W', 'A']
Damen/Milwaukee
['U', 'W', 'A']
East 63rd-Cottage Grove
['U', 'W', 'A']
Austin-Lake
['U', 'W', 'A']
Cumberland
['U', 'W', 'A']
35-Bronzeville-IIT
['U', 'W', 'A']
Medical Center
['U', 'W', 'A']
Grand/State
['U', 'W', 'A']
Linden
['U', 'W', 'A']
Skokie
['U', 'W', 'A']
95th/Dan Ryan
['U', 'W', 'A', nan]


TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [nan] of <class 'float'>

In [ ]:
dan = ride_df[ride_df['stationname']=='95th/Dan Ryan']
dan[dan['daytype']=='W'].mean()

In [ ]:
dan.daytype.unique()

In [ ]:
summary = {}

In [ ]:
morse.summary